# Build a Retrieval Augmented Generation (RAG) Application with Semantic Kernel

## Workshop Overview

Welcome to this hands-on tutorial where you'll learn how to build a **Retrieval Augmented Generation (RAG)** application using **Microsoft Semantic Kernel**!

### What You'll Learn

This notebook takes a progressive approach, building your understanding step-by-step:

1. **Basic LLM Interaction** - Start with simple prompting using Semantic Kernel
2. **Prompt Engineering** - Learn to structure prompts with SK templates
3. **Function Calling** - Understand SK's plugin architecture
4. **Document Loading & Splitting** - Prepare data for retrieval
5. **Vector Embeddings & Storage** - Create searchable knowledge bases with InMemoryStore
6. **Complete RAG Implementation** - Put it all together with SK functions

### What is RAG?

**Retrieval Augmented Generation (RAG)** is one of the most powerful applications enabled by LLMs. It allows AI to answer questions using **specific source information** rather than relying solely on its training data.

Think of it like giving an AI assistant access to a specialized library - it can look up relevant information before answering your questions!

### Why Semantic Kernel?

**Semantic Kernel (SK)** is Microsoft's lightweight SDK for integrating AI services into applications. Key benefits:

- ✅ **Cross-platform** - Works with Python, C#, and Java
- ✅ **Plugin architecture** - Modular, reusable components
- ✅ **Enterprise-ready** - Built for production use
- ✅ **Multi-AI support** - Works with Azure OpenAI, OpenAI, and more
- ✅ **Function calling** - Natural integration with tools and data sources

### The RAG Architecture

A typical RAG application has **two main components**:

1. **Indexing** (Offline) - Preparing your knowledge base
   - Load documents from various sources
   - Split large documents into chunks
   - Create embeddings and store in a vector database

2. **Retrieval & Generation** (Runtime) - Answering questions
   - Retrieve relevant documents based on the query
   - Generate answers using retrieved context

---

Let's get started! 🚀

## Step 0: Environment Setup

Before we begin, let's install all the required packages for **Semantic Kernel**.

### What We'll Install:

- **semantic-kernel** - Microsoft's AI orchestration SDK
- **python-dotenv** - Environment variable management
- **faiss-cpu** - Facebook AI Similarity Search for vector storage
- **beautifulsoup4** - HTML parsing for web content loading

Run the cell below to install all dependencies:

In [ ]:
%pip install semantic-kernel python-dotenv
%pip install faiss-cpu
%pip install beautifulsoup4
%pip install aiohttp

## Step 1: Basic LLM Interaction with Semantic Kernel

Let's start with the fundamentals - setting up Semantic Kernel and making a simple request.

### What's Happening Here?

1. **Import Semantic Kernel** - Load the SK SDK
2. **Load environment variables** - Read Azure OpenAI credentials from `.env` file
3. **Create a Kernel** - The central orchestrator in SK
4. **Add AI Service** - Connect to Azure OpenAI
5. **Make a simple query** - Ask a question directly

### Key Concept: The Kernel

In Semantic Kernel, the **Kernel** is the central orchestrator that:
- Manages AI services (chat, embeddings)
- Hosts plugins and functions
- Handles prompt execution
- Coordinates function calling

This is the **simplest** way to interact with an LLM using SK, but notice:
- ❌ No specialized knowledge
- ❌ No structured prompting
- ❌ Limited control over format
- ❌ Answers only from training data

Let's see it in action:

In [ ]:
import os
from dotenv import load_dotenv

# Import Semantic Kernel components
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables
load_dotenv()

# Create the Kernel - the central orchestrator
kernel = Kernel()

# Add Azure OpenAI Chat Completion service
kernel.add_service(
    AzureChatCompletion(
        service_id="chat",
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    )
)

print("✓ Semantic Kernel initialized with Azure OpenAI")

### Test Basic LLM Call

Let's ask the LLM a simple question using `invoke_prompt`. Notice how it answers from its general training knowledge:

In [ ]:
# Simple prompt invocation
answer = await kernel.invoke_prompt("What is machine learning?")

In [ ]:
print(f"Answer: {answer}")
print(f"Type: {type(answer)}")

### Understanding the Response

Semantic Kernel's `invoke_prompt` returns a `FunctionResult` object containing:
- **value** - The actual text answer from the LLM
- **metadata** - Information about the execution

The LLM gives a general answer based on its training, but it doesn't have access to:
- ✗ Up-to-date information
- ✗ Your company's internal documentation  
- ✗ Specific domain knowledge from your documents

**This is where RAG becomes powerful!** But first, let's learn about prompt engineering with SK...

## Step 2: Prompt Engineering with Semantic Kernel

Now let's improve our LLM interactions using **SK's prompt template system**!

### Why Use Prompt Templates in SK?

Semantic Kernel provides powerful prompt engineering capabilities:

- ✅ **Variable Substitution** - Use `{{$variable}}` for dynamic content
- ✅ **Function Calling** - Embed function results with `{{plugin.function}}`
- ✅ **Multi-shot Prompting** - Structure conversations easily
- ✅ **Prompt Configuration** - Separate prompts from code
- ✅ **Reusability** - Create prompt libraries

### SK Template Syntax

Semantic Kernel uses a special syntax:
- `{{$input}}` - Access input variables
- `{{$variable_name}}` - Access named variables
- `{{plugin.function}}` - Call SK functions
- Multi-line templates with proper formatting

This gives you:
- **Consistency** - Same structure every time
- **Reusability** - One template, many uses
- **Maintainability** - Update once, affects all uses

### Example 1: Simple Variable Substitution

Let's create a template with variables using SK's `{{$variable}}` syntax:

In [ ]:
from semantic_kernel.functions import KernelArguments

# SK template using {{$variable}} syntax
simple_template = """
You are a helpful AI assistant named {{$bot_name}}.
User question: {{$question}}

Please provide a helpful answer.
"""

# Invoke with arguments
result = await kernel.invoke_prompt(
    function_name="simple_prompt",
    plugin_name="PromptDemo",
    prompt=simple_template,
    arguments=KernelArguments(
        bot_name="SKBot",
        question="What is Semantic Kernel?"
    )
)

print(result)

Notice how SK filled in the `{{$bot_name}}` and `{{$question}}` placeholders!

### Example 2: Structured Technical Writing Template

Here's a more sophisticated template for a specific task:

In [ ]:
# Technical documentation template
tech_writer_template = """
You are a world-class technical documentation writer.

Topic: {{$topic}}

Please write a clear, concise explanation suitable for developers.
Use three sentences maximum and keep the answer concise.
"""

result = await kernel.invoke_prompt(
    function_name="tech_writer",
    plugin_name="PromptDemo",
    prompt=tech_writer_template,
    arguments=KernelArguments(topic="What is RAG (Retrieval Augmented Generation)?")
)

print(result)

## Step 3: SK Plugins and Functions

Now let's learn about **Semantic Kernel's plugin architecture** - a key differentiator from other frameworks!

### What are SK Plugins?

**Plugins** in Semantic Kernel are collections of functions that extend the kernel's capabilities:

- ✅ **Native Functions** - Python functions decorated with `@kernel_function`
- ✅ **Prompt Functions** - AI-powered functions defined as prompts
- ✅ **Reusable** - Share across projects
- ✅ **Composable** - Combine multiple functions
- ✅ **Auto-invokable** - LLM can call them automatically

### Why is This Powerful?

Unlike simple chaining, SK's plugin system allows:
- **Modular Design** - Organize functions by domain
- **Function Calling** - LLM automatically chooses which function to use
- **State Management** - Functions can maintain state
- **Tool Integration** - Connect to APIs, databases, etc.

Let's create our first plugin!

### Example: Create a Simple Native Plugin

Let's create a plugin with a native Python function:

In [ ]:
from semantic_kernel.functions import kernel_function

# Create a simple plugin class
class TextPlugin:
    @kernel_function(
        name="get_word_count",
        description="Counts the number of words in a text"
    )
    def get_word_count(self, text: str) -> str:
        word_count = len(text.split())
        return f"The text contains {word_count} words."
    
    @kernel_function(
        name="uppercase",
        description="Converts text to uppercase"
    )
    def to_uppercase(self, text: str) -> str:
        return text.upper()

# Add the plugin to the kernel
kernel.add_plugin(TextPlugin(), plugin_name="TextPlugin")

print("✓ TextPlugin added to kernel")

### Test the Plugin Functions

Now let's call our plugin functions directly:

In [ ]:
# Invoke plugin functions directly
test_text = "Semantic Kernel makes AI development easier and more organized"

word_count_result = await kernel.invoke(
    function_name="get_word_count",
    plugin_name="TextPlugin",
    arguments=KernelArguments(text=test_text)
)

uppercase_result = await kernel.invoke(
    function_name="uppercase",
    plugin_name="TextPlugin",
    arguments=KernelArguments(text=test_text)
)

print(f"Word Count: {word_count_result}")
print(f"Uppercase: {uppercase_result}")

## Step 4: Document Loading & Text Splitting

Now we get to the heart of RAG! Let's learn how to **load and split documents** for retrieval.

### The RAG Indexing Pipeline

Before we can retrieve, we need to **index our documents**:

```
Document → Load → Split → Embed → Store in Vector Database
```

This happens **offline** (once) to prepare your knowledge base.

### What We'll Build

1. **Load** document content (we'll use mock data for this tutorial)
2. **Split** it into chunks using a text splitter
3. **Embed** each chunk (convert to vectors) - Next step
4. **Store** in a vector database using SK's InMemoryStore

### Why Split Documents?

- LLMs have limited context windows
- Smaller chunks = more precise retrieval
- Better matching between queries and relevant content
- Easier to attribute sources

Let's start!

### Step 4.1: Create Mock Documents

For this tutorial, we'll use mock documents about AI topics (similar to the reference implementation):

In [ ]:
# Mock paragraph data - sample documents about AI topics
MOCK_DOCUMENTS = """
Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans. AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals.

Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and statistical models that enable computer systems to improve their performance on a specific task through experience. Deep learning, a subset of machine learning, uses neural networks with multiple layers to progressively extract higher-level features from raw input.

Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language. NLP is used to apply algorithms to identify and extract the natural language rules such that the unstructured language data is converted into a form that computers can understand.

Computer vision is an interdisciplinary scientific field that deals with how computers can gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to understand and automate tasks that the human visual system can do. Computer vision tasks include methods for acquiring, processing, analyzing and understanding digital images.

Reinforcement learning is an area of machine learning concerned with how intelligent agents ought to take actions in an environment in order to maximize the notion of cumulative reward. Reinforcement learning is one of three basic machine learning paradigms, alongside supervised learning and unsupervised learning.

Neural networks are computing systems inspired by the biological neural networks that constitute animal brains. Such systems learn to perform tasks by considering examples, generally without being programmed with task-specific rules. For instance, in image recognition, they might learn to identify images that contain cats by analyzing example images.

Generative AI refers to artificial intelligence systems capable of generating text, images, or other media in response to prompts. Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics. Examples include large language models like GPT and image generation models like DALL-E.

The Transformer architecture is a neural network architecture that has become the foundation for many modern AI models. It uses self-attention mechanisms to process input data in parallel, making it highly efficient for tasks like language translation and text generation. Transformers have revolutionized natural language processing since their introduction in 2017.

Retrieval Augmented Generation (RAG) is a technique that enhances large language models by retrieving relevant information from external knowledge sources before generating responses. RAG combines the power of pre-trained language models with the ability to access up-to-date, domain-specific information, making AI responses more accurate and grounded in facts.

Semantic Kernel is Microsoft's open-source SDK that helps developers integrate AI services into their applications. It provides a plugin-based architecture that makes it easy to combine multiple AI models, prompts, and native code into unified workflows. Semantic Kernel supports function calling, memory management, and orchestration of complex AI pipelines.
"""

print(f"Loaded document with {len(MOCK_DOCUMENTS)} characters")

### Step 4.2: Create a Text Splitter Function

We need to split our document into manageable chunks. Let's create a simple paragraph-based splitter:

In [ ]:
def split_into_paragraphs(text: str) -> list[str]:
    """Split text into paragraphs by empty lines."""
    paragraphs = [p.strip() for p in text.strip().split('\n\n') if p.strip()]
    return paragraphs

# Split the documents
paragraphs = split_into_paragraphs(MOCK_DOCUMENTS)
print(f"Split document into {len(paragraphs)} paragraphs\n")

# Show first paragraph as example
print("Example paragraph:")
print(f"{paragraphs[0][:200]}...")

## Step 5: Vector Embeddings & InMemoryStore

Now let's convert our text chunks into **vector embeddings** and store them in Semantic Kernel's **InMemoryStore**.

### What are Vector Embeddings?

Embeddings convert text into numerical vectors (lists of numbers) that capture semantic meaning:
- Similar concepts have similar vectors
- Enables semantic search (meaning-based, not keyword-based)
- Typically 1536 dimensions for Azure OpenAI's text-embedding models

### SK's InMemoryStore

Semantic Kernel provides `InMemoryCollection` for vector storage:
- ✅ **Simple setup** - No external database needed
- ✅ **Built-in search** - Semantic similarity search included
- ✅ **Dataclass-based** - Type-safe document models
- ✅ **Auto-embedding** - Automatic vector generation

Let's set it up!

### Step 5.1: Setup Azure Text Embedding Service

First, let's add the embedding service to our kernel:

In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding

# Add embedding service to kernel
text_embedding = AzureTextEmbedding(
    service_id="embedding",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")
)

kernel.add_service(text_embedding)
print("✓ Azure Text Embedding service added to kernel")

### Step 5.2: Define Document Model with Vector Store Decorator

SK uses dataclasses with special decorators to define document models:

In [ ]:
from dataclasses import dataclass
from typing import Annotated
from semantic_kernel.data.vector import VectorStoreField, vectorstoremodel

@vectorstoremodel(collection_name="documents")
@dataclass
class DocumentParagraph:
    """Data model for storing document paragraphs with embeddings"""
    id: Annotated[str, VectorStoreField("key")]
    text: Annotated[str, VectorStoreField("data")]
    embedding: Annotated[
        list[float] | None,
        VectorStoreField(
            "vector",
            dimensions=1536,
            embedding_generator=text_embedding
        ),
    ] = None

print("✓ DocumentParagraph model defined")
print("  - Fields: id (key), text (data), embedding (vector)")
print("  - Auto-embedding enabled via text_embedding service")
print("  - Dimensions: 1536 (Azure OpenAI text-embedding-ada-002)")

## Step 6: Complete RAG Implementation with Semantic Kernel

Now let's put it all together! We'll:
1. Create an InMemoryCollection
2. Index our documents with embeddings
3. Create a search function (recall)
4. Build RAG queries

### The Complete RAG Flow

```
User Question → Search Vector Store → Retrieve Relevant Docs → Generate Answer with Context
```

This is where Semantic Kernel really shines with its **unified approach**!

### Step 6.1: Create InMemoryCollection and Index Documents

Let's create the collection, index our paragraphs, and set up the search function:

In [ ]:
from semantic_kernel.connectors.in_memory import InMemoryCollection

# Create the in-memory collection
collection = InMemoryCollection(record_type=DocumentParagraph)

# Initialize the collection
await collection.ensure_collection_exists()

# Create DocumentParagraph objects and upsert into collection
document_items = [
    DocumentParagraph(id=f"para_{i}", text=para)
    for i, para in enumerate(paragraphs)
]

await collection.upsert(document_items)

print(f"✓ Loaded {len(paragraphs)} paragraphs into the vector store")
print("✓ Documents successfully indexed with embeddings")

### 🔍 Understanding Vector Embeddings (Optional Demo)

Before we move on to RAG queries, let's take a moment to understand what just happened with vector embeddings!

**What are vectors?**
- Each paragraph was converted into a list of 1,536 numbers
- These numbers capture the *semantic meaning* of the text
- Similar texts have similar vectors

Let's see this in action with a simple similarity comparison:

In [ ]:
import numpy as np

def cosine_similarity(vec_a: list[float], vec_b: list[float]) -> float:
    """Calculate cosine similarity between two vectors (0-1, higher = more similar)"""
    return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

# Test sentences with different similarity levels
test_sentences = [
    "Machine learning uses algorithms to learn from data",
    "Algorithms enable computers to learn from experience",  # Similar meaning
    "Pizza and pasta are Italian foods"  # Completely different topic
]

# Generate embeddings for test sentences
print("Generating embeddings for test sentences...\n")
test_embeddings = []
for sentence in test_sentences:
    response = await text_embedding.generate_embeddings([sentence])
    test_embeddings.append(response[0])

# Compare first sentence with the other two
base_sentence = test_sentences[0]
print(f"Base sentence: '{base_sentence}'\n")
print("Similarity comparisons:")
print("-" * 80)

for i in range(1, len(test_sentences)):
    similarity = cosine_similarity(test_embeddings[0], test_embeddings[i])
    print(f"📊 Comparison sentence: '{test_sentences[i]}'")
    print(f"   Similarity score: {similarity:.4f}")
    print()

print("💡 Notice: Similar meanings have higher scores (closer to 1.0)")
print("   Even when different words are used!")

**What Just Happened?**

1. We converted three sentences into vectors (1,536 numbers each)
2. We used **cosine similarity** to measure how "close" the meanings are
3. The second sentence scores high (~0.85-0.95) because it means nearly the same thing
4. The third sentence scores low (~0.60-0.75) because it's about a completely different topic

**This is how vector search works:**
- Your query gets converted to a vector
- The system finds the most similar document vectors
- Similar meaning = high similarity score = relevant results!

Now let's test our actual RAG system with real queries! 🚀

### Step 6.2: Create Search Function (Recall)

SK's InMemoryCollection can automatically create a search function that we add as a plugin:

In [ ]:
# Create a search function for the collection
kernel.add_function(
    "memory",
    collection.create_search_function(
        function_name="recall",
        description="Recalls information from the document collection about AI topics.",
        string_mapper=lambda x: x.record.text,  # Extract text from search results
    ),
)

print("✓ Search function 'memory.recall' added to kernel")
print("  - Function automatically performs vector similarity search")
print("  - Returns relevant document chunks based on query")

### Step 6.3: Template-Based RAG Query

Let's test our first RAG query using SK's template syntax with `{{memory.recall}}`:

In [ ]:
# Simple RAG query using template syntax
query = "What are the main benefits of prompt engineering?"

rag_prompt = f"""
Use the information below to answer the question.

Context: {{{{memory.recall '{query}'}}}}

Question: {query}

Answer:"""

result = await kernel.invoke_prompt(
    function_name="rag_query",
    plugin_name="rag",
    prompt=rag_prompt
)

print(f"Query: {query}")
print(f"\nAnswer:\n{result}")

**What Just Happened?**

When we invoked the prompt:
1. SK saw `{{memory.recall 'query'}}` and executed the recall function
2. The recall function searched the InMemoryCollection for relevant chunks
3. Those chunks were injected into the Context section of the prompt
4. The LLM used that context to answer the question

Let's try another query:

In [ ]:
# Another RAG query
query2 = "How do AI agents differ from traditional chatbots?"

rag_prompt2 = f"""
Use the information below to answer the question.

Context: {{{{memory.recall '{query2}'}}}}

Question: {query2}

Answer:"""

result2 = await kernel.invoke_prompt(
    function_name="rag_query2",
    plugin_name="rag",
    prompt=rag_prompt2
)

print(f"Query: {query2}")
print(f"\nAnswer:\n{result2}")

---

## Step 7: Auto Function Calling (Advanced)

**What We're About to Build:**

So far we've explicitly called `{{memory.recall}}` in our prompts. But Semantic Kernel can automatically determine when to call functions based on the user's question!

This is the foundation of **agentic behavior** - the LLM decides which tools to use.

### Step 7.1: Enable Auto Function Calling

We'll configure the kernel to automatically call functions when needed:

In [ ]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

# Create execution settings with auto function calling enabled
execution_settings = kernel.get_prompt_execution_settings_from_service_id("chat")
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

print("✅ Auto function calling enabled!")
print("The LLM will now automatically decide when to call the memory.recall function.")

### Step 7.2: Ask Questions Without Explicit Function Calls

Now let's ask questions naturally - the LLM will automatically search the documents when needed:

In [ ]:
# Ask a question that requires document search
user_question = "What are the key capabilities of AI agents?"

# Simple prompt - no explicit function calls!
simple_prompt = f"Answer this question: {user_question}"

# The LLM will automatically call memory.recall if needed
result = await kernel.invoke_prompt(
    prompt=simple_prompt,
    function_name="auto_rag",
    plugin_name="rag",
    arguments={"settings": execution_settings}
)

print(f"Question: {user_question}")
print(f"\nAnswer:\n{result}")

**What Happened Behind the Scenes?**

The LLM analyzed the question, realized it needed more information, automatically called `memory.recall("key capabilities of AI agents")`, and then used the results to answer!

Let's try more questions:

In [ ]:
# Test multiple questions with auto function calling
questions = [
    "Explain vector embeddings in simple terms",
    "What is the difference between RAG and fine-tuning?",
    "How does prompt engineering improve AI responses?"
]

for question in questions:
    result = await kernel.invoke_prompt(
        prompt=f"Answer concisely: {question}",
        function_name="auto_rag_multi",
        plugin_name="rag",
        arguments={"settings": execution_settings}
    )
    print(f"Q: {question}")
    print(f"A: {result}\n")
    print("-" * 80)

---

## 🎓 Conclusion: What You've Learned

Congratulations! You've built a complete RAG system with Semantic Kernel from the ground up.

### Journey Recap:

**Step 0: Environment Setup**
- Installed Semantic Kernel and dependencies

**Step 1: Basic LLM Interaction**
- Created a kernel
- Added Azure OpenAI chat service
- Made your first AI call

**Step 2: Prompt Engineering**
- Used SK template syntax (`{{$variable}}`)
- Created reusable prompt templates
- Structured better prompts

**Step 3: Plugins & Functions**
- Built a TextPlugin with `@kernel_function`
- Registered functions with the kernel
- Called functions in prompts

**Step 4: Document Processing**
- Loaded and split documents into chunks
- Prepared text for embeddings

**Step 5: Vector Embeddings**
- Set up Azure Text Embedding service
- Created document models with `@vectorstoremodel`
- Understood how semantic search works

**Step 6: RAG Implementation**
- Created InMemoryCollection
- Indexed documents with embeddings
- Built search functions
- Performed template-based RAG queries

**Step 7: Auto Function Calling**
- Enabled `FunctionChoiceBehavior.Auto()`
- Let the LLM decide when to search
- Built true agentic behavior!

### 🔑 Key Semantic Kernel Concepts:

| Concept | Purpose |
|---------|---------|
| **Kernel** | Central orchestrator managing services, plugins, and functions |
| **Plugins** | Collections of functions organized by purpose |
| **@kernel_function** | Decorator to expose Python functions to the kernel |
| **Template Syntax** | `{{$variable}}` and `{{plugin.function}}` for dynamic prompts |
| **InMemoryCollection** | Built-in vector store with automatic search |
| **@vectorstoremodel** | Decorator to define document models for vector storage |
| **FunctionChoiceBehavior** | Controls how the LLM selects and calls functions |

### 🆚 Semantic Kernel vs LangChain

**Semantic Kernel:**
- ✅ Microsoft-native, tight Azure integration
- ✅ Plugin architecture with decorators
- ✅ Built-in vector stores
- ✅ Strong typing and async-first
- ⚠️ Smaller community than LangChain

**LangChain:**
- ✅ Larger ecosystem and community
- ✅ More integrations (100+ vector DBs, LLMs)
- ✅ Rich documentation and examples
- ⚠️ Can be complex with many abstractions

**Both are excellent choices!** Pick based on your ecosystem and team preferences.

### 📚 Resources & Next Steps

**Official Documentation:**
- [Semantic Kernel Docs](https://learn.microsoft.com/semantic-kernel/)
- [Semantic Kernel GitHub](https://github.com/microsoft/semantic-kernel)
- [Azure OpenAI Service Docs](https://learn.microsoft.com/azure/ai-services/openai/)

**Advanced Topics to Explore:**
1. **Planners** - Multi-step reasoning and planning
2. **Memory Connectors** - Persistent storage (Redis, PostgreSQL, Qdrant)
3. **Streaming** - Real-time response streaming
4. **Multi-Agent Systems** - Coordinating multiple AI agents
5. **Custom Connectors** - Integrate any LLM or service

**Practice Exercises:**
1. Add more documents to the collection
2. Create a plugin with multiple related functions
3. Build a chat interface that maintains conversation history
4. Integrate a different vector database (Qdrant, Redis)
5. Add function calling for non-RAG tasks (weather, calculations, etc.)

---

### 🙏 Thank You!

You now have the foundation to build production-ready RAG systems with Semantic Kernel. Keep experimenting, and happy coding! 🚀